In [3]:
# importazione librerie necessarie
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.datasets import mnist
from tqdm.notebook import trange
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import Adam
%matplotlib inline

In [4]:
# funzione per caricare i dati del MNIST
def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    
    # convert shape of x_train from (60000, 28, 28) to (60000, 784) 
    # 784 columns per row
    x_train = x_train.reshape(60000, 784)
    return (x_train, y_train, x_test, y_test)

(X_train, y_train,X_test, y_test)=load_data()
print(X_train.shape)

11490434/11490434 [==============================] - 1s 0us/step
(60000, 784)


In [5]:
# ottimizzatore
# con beta_1 non standard (meno "inerzia") e lr un quarto
# di quello di default
def adam_optimizer():
    return Adam(learning_rate=0.00005, beta_1=0.65)

In [6]:
# generatore
def create_generator():
    generator=Sequential()
    generator.add(Dense(units=256, input_dim=100, activation='relu'))
    
    generator.add(Dense(units=512, activation='relu'))

    generator.add(Dense(units=1024, activation='relu'))
    
    generator.add(Dense(units=784, activation='tanh'))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator

g = create_generator()
g.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               25856     
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 1024)              525312    
                                                                 
 dense_3 (Dense)             (None, 784)               803600    
                                                                 
Total params: 1,486,352
Trainable params: 1,486,352
Non-trainable params: 0
_________________________________________________________________


In [7]:
# discriminatore
def create_discriminator():
    discriminator=Sequential()
    discriminator.add(Dense(units=1024, input_dim=784, activation='relu'))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(units=512, activation='relu'))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(units=256, activation='relu'))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator

d = create_discriminator()
d.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 1024)              803840    
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_5 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               131328    
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 1,460,225
Trainable params: 1,460,225
No

In [8]:
# GAN (con Adam parametri di default)
def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)

    # Il discriminatore è addestrato a dire:
    # 0.9: real images
    # 0: generated images
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return gan

gan = create_gan(d, g)
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 784)               1486352   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 1460225   
                                                                 
Total params: 2,946,577
Trainable params: 1,486,352
Non-trainable params: 1,460,225
_________________________________________________________________


In [9]:
# preview immagini
def plot_generated_images(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    #plt.savefig('data/gan_generated_images/gan_generated_image_%d.png' %epoch)

In [10]:
# contatore epoche di addestramento
# utile per il log durante il training
epoche_addestramento = 0

In [11]:
# training
def training(epochs, batch_size, epoche_addestramento):
    #Loading the data
    (X_train, y_train, X_test, y_test) = load_data()
    
    # Creazione modello:
    # prima generatore e discriminatore quindi la GAN
    generator = create_generator()
    discriminator = create_discriminator()
    gan = create_gan(discriminator, generator)
    tot_epoche = epoche_addestramento + epochs
    for e in range(1, epochs + 1):
        losses = 0
        epoca_attuale = e + epoche_addestramento
        t = trange(batch_size, desc='', leave=True)
        for b in t:
            msg = 'Epoca ' + str(epoca_attuale) + '/' + str(tot_epoche)
            msg += ' - Batch ' + str(b + 1) + '/' + str(batch_size)
            msg += ' - Loss ' + str(losses / (b + 1))[:6] + ' '
            t.set_description(msg)
            t.refresh()
            # generazione di "disturbo" come input per il generatore
            noise = np.random.normal(0,1, [batch_size, 100])
            
            # generazione di immagini fake del MNIST
            generated_images = generator.predict(noise)
            
            # recupero di un subset casuale di immaigni del MNIST
            image_batch = X_train[np.random.randint(low=0, high=X_train.shape[0], size=batch_size)]
            
            # costruzione di batch di immagini reali e generate 
            X = np.concatenate([image_batch, generated_images])
            
            # etichette per le immagini reali e generate
            # Addestro il discriminatore a dire:
            # 0.9: real images
            # 0: generated images

            y_dis = np.zeros(2 * batch_size)
            y_dis[:batch_size] = 0.9
            
            # training del discriminatore su immagini reali e generate
            # prima dell'addestramento della GAN 
            discriminator.trainable = True
            losses += discriminator.train_on_batch(X, y_dis)
            
            # Creazione di un dataset utile al generatore
            noise = np.random.normal(0, 1, [batch_size, 100])
            y_gen = np.ones(batch_size)
            
            # addestramento della GAN tramite il blocco 
            # dell'adddestramento del discriminatore
            discriminator.trainable = False
            
            # addestramento della GAN (discriminatore bloccato,
            # generatore che si addestra ad "ingannare" il
            # discriminatore).
            # Le label sono tutte 1, in quanto tutte le immagini
            # passate sono generate
            gan.train_on_batch(noise, y_gen)
        
        # ogni tot epoche vengono create delle immagini per
        # visualizzare i file generati dal generatore
        if epoca_attuale == 1 or epoca_attuale == 5 or epoca_attuale == 10 or epoca_attuale % 20 == 0:
            plot_generated_images(e, generator)
    
    return epoche_addestramento + epochs

In [12]:
A = np.concatenate([ ['a']*4, ['b']*4])
print(A)

print(len(A))

l = len(A)//2

B = np.zeros(2 * l )
B[:l ] = 1

print(B)

['a' 'a' 'a' 'a' 'b' 'b' 'b' 'b']
8
[1. 1. 1. 1. 0. 0. 0. 0.]


In [13]:
# addestramento su 400 epoche
epoche = 400
epoche_addestramento = training(epochs=epoche, batch_size=128, epoche_addestramento=epoche_addestramento)

Output hidden; open in https://colab.research.google.com to view.